In [36]:
import get_price.get_futures_price as gfp
import pandas as pd
data_out = gfp.get_futures_price_preloaded(ticker_head='EC')

data_out.rename(columns={'open_price': 'open', 'high_price': 'high', 'low_price': 'low', 'close_price': 'close'}, inplace=True)


contract_data = data_out[data_out['cont_indx']==201503]

In [19]:
contract_data.columns

Index(['settle_date', 'ticker', 'ticker_head', 'ticker_month', 'ticker_year',
       'cal_dte', 'tr_dte', 'open', 'high', 'low', 'close', 'volume',
       'cont_indx', 'change1', 'change1_instant', 'high1_instant',
       'low1_instant', 'change2', 'change5', 'change10', 'change20',
       'change_5', 'change_1'],
      dtype='object')

In [21]:
import pandas as pd

momentum_window_list = [3, 6, 9, 12]

for i in range(len(momentum_window_list)):
    contract_data['momentum_' + str(momentum_window_list[i])] = \
    100*contract_data['close']/contract_data['close'].shift(momentum_window_list[i])


In [26]:
import signals.technical_indicators as ti

# stochastic

stochastic_window_list = [3, 6, 9, 12]

for i in range(len(stochastic_window_list)):

    contract_data = ti.stochastic(data_frame_input = contract_data,p1 = stochastic_window_list[i],p2=3, p3=3)
    contract_data['stochastic_'+ str(stochastic_window_list[i])] = contract_data['D1']

In [33]:
# williams accumulation distribution line

contract_data['close_1'] = contract_data['close'].shift(1)



contract_data['trh'] = contract_data[['close_1', 'high']].max(axis=1)
contract_data['trl'] = contract_data[['close_1', 'low']].min(axis=1)

contract_data['price_move'] = 0

up_day = contract_data['close']>contract_data['close_1']
contract_data['price_move'].loc[up_day] = contract_data['close'].loc[up_day]-contract_data['trl'].loc[up_day]

down_day = contract_data['close']<contract_data['close_1']
contract_data['price_move'].loc[down_day] = contract_data['close'].loc[down_day]-contract_data['trh'].loc[down_day]
contract_data['AD'] = contract_data['price_move']*contract_data['volume']
contract_data['Williams_AD'] = pd.rolling_sum(contract_data['AD'].cumsum(),15)

In [37]:
# accumulation distribution oscillator

contract_data['mfm'] = (2*contract_data['close']-contract_data['low']-contract_data['high'])/(contract_data['high']-contract_data['low'])
contract_data['mfv'] = 
